## Function

#### Read the average of channel 3 and export that data

In [2]:
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import Tag_Database as Tags
import Master as M
import time
seaborn.set()

from pymodbus.client.sync import ModbusTcpClient
from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.payload import BinaryPayloadBuilder
from pymodbus.constants import Endian

In [182]:
RM = pyvisa.ResourceManager()
RM.list_resources()
SG = RM.open_resource('GPIB0::1::INSTR')
OS = RM.open_resource('GPIB0::10::INSTR')

In [183]:
def freq(Device):
    
    frequency = Device.query("freq:cw?")
    
    return float(frequency)

def write_frequency(Device, Value, Units = "MHZ"):
    
    Device.write("freq:cw {} {}".format(Value,Units))
    
    return

def read_mv(OSCOPE):
    return 1000*float(OS.query("MEASU:MEAS1:VAL?").split(' ')[1].strip("\n"))

def read_v(OSCOPE):
    return float(OS.query("MEASU:MEAS1:VAL?").split(' ')[1].strip("\n"))

def mean_meas(OSCOPE, CHANNEL, MEASUREMENT, SCALE = False):
    
    if type(CHANNEL) and type(MEASUREMENT) != int:
        print("Must be integers!")
    
    if not SCALE == False:
        OS.write("CH3:SCA {:.1E}".format())
        
    OS.write("MEASU:MEAS{}:SOURCE1 CH{}".format(MEASUREMENT, CHANNEL))
    OS.write("MEASU:MEAS{}:UNI v".format(MEASUREMENT))
    OS.write("MEASU:MEAS{}:STATE ON".format(MEASUREMENT))
    time.sleep(1)
    return OS.query("MEASU:meas{}:val?".format(MEASUREMENT))

In [187]:
SG.write("*wai")

(6, <StatusCode.success: 0>)

In [184]:
OS.write("CH3:SCA 5.0E-3")
OS.write("MEASU:MEAS1:SOURCE1 CH3")
OS.write("MEASU:MEAS1:UNI v")
OS.write("MEASU:MEAS1:STATE ON")
time.sleep(1)
OS.query("MEASU:meas1:val?")


temp_list = []


for i in range(25):
    
    temp_list.append(1000*float(OS.query("MEASU:MEAS1:VAL?").split(' ')[1].strip("\n")))
    #temp_list.append(1000*float(OS.query("CURSor:VBArs:HPOS1?").split(' ')[1].strip("\n")))
    time.sleep(0.5)
    
np.array(temp_list)

array([ 7.1768 ,  3.0143 ,  0.63882, -4.2157 , -5.4255 , -6.7156 ,
       -1.903  ,  0.50224,  2.876  ,  7.1448 ,  6.0883 ,  3.7033 ,
       -1.107  , -3.5166 , -6.8768 , -4.959  , -2.5899 ,  2.1957 ,
        4.6602 ,  7.0233 ,  4.3864 ,  2.0007 , -0.48883, -5.2434 ,
       -6.8416 ])

In [ ]:
Start_Freq = float(SG.query("freq:cw?"))
SG.write("FM:STAT OFF")

print("Beginning frequency modulation in 5 seconds")
time.sleep(5)
print("Beginning modulation")
SG.write("FM:STAT ON")

Start_Time = time.time()
Ups = 0
Downs = 0
while time.time() - Start_Time < 100:
    if Read_Bool(Client, 30001) == True:
        Ups += 1
        Downs = 0
        if Ups > 10:
            temp_freq = freq(SG)
            write_frequency(SG, temp_freq + 10000 ,"HZ")
            print("Walking frequency up")
            time.sleep(0.1)
    if Read_Bool(Client, 30003) == True:
        Downs += 1
        Ups = 0
        if Downs > 10:
            temp_freq = freq(SG)
            write_frequency(SG, temp_freq - 10000 ,"HZ")
            print("Walking frequency down")
            time.sleep(0.1)
    time.sleep(0.01)

print("\n\n\nModulation Over")

In [191]:
def write_frequency(Device, Value, Units = "MHZ"):
    
    Device.write("freq:cw {} {}".format(Value,Units))
    Device.write("*wai")
    
    return
temp = freq(SG)
write_frequency(SG, temp + 200, "HZ")